In [3]:
import pickle
import os
import nltk
import re
import numpy as np
from string import punctuation
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_list = stopwords.words('english')
ps = PorterStemmer()

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to C:\Users\Sai
[nltk_data]     Nadkarni\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
crawledList = []
pageTitles = []
webData = []

with open('pageList.txt', "rb") as out:
    crawledList = pickle.load(out)
out.close()

with open('pageTitles.txt', "rb") as out:
    pageTitles = pickle.load(out)
out.close()


with open('webData.txt', "rb") as out:
    webData = pickle.load(out)
out.close()

with open('page_rank.pkl', 'rb') as out:
    pageRank = pickle.load(out)
out.close()

In [5]:
def cleaner(data):
    cleanData = []
    for file in data:
        temp = file.split()
        temp = [ps.stem(word) for word in temp]
        temp = [item for item in temp if item not in stop_list]
        temp = [item for item in temp if len(item) > 2]
        temp = (" ").join(temp)
        for i in punctuation:
            temp = temp.replace(i, "")
        cleanData.append(temp)
    return cleanData

In [6]:
print("Cleaning and pre-processing webpage data, this may take 20-30 seconds")
cleanWebData = cleaner(webData)
print("Webpage data pre processing complete.")

Cleaning and pre-processing webpage data, this may take 15-20 seconds
Webpage data pre processing complete.


In [7]:
vectorizer = TfidfVectorizer()
tfidfs = vectorizer.fit_transform(cleanWebData)

In [8]:
def queryTokenize(query):
    temp = query.lower()
    temp = temp.split()
    temp = [ps.stem(word) for word in temp]
    temp = [item for item in temp if item not in stop_list]
    temp = [item for item in temp if len(item) > 2]
    temp = (" ").join(temp)
    for i in punctuation:
        temp = temp.replace(i, "")
    return temp.split()

In [9]:
def correlateTitleURL(length):
    pageNums = []
    for i in range(length):
        pageNums.append([int(bestFirstList[i][1]), 0.4 * bestFirstList[i][0] + 0.6 * pageRank.get(crawledList[int(bestFirstList[i][1])])])
    
    pageNums = sorted(pageNums, key= lambda x: x[1], reverse=True)
    
    for item in range(length):
        print(item + 1, "-" + pageTitles[pageNums[item][0]] + "\nhttps://" + crawledList[pageNums[item][0]] + "\n")

In [12]:
query = str(input("Enter a search query: "))
print('\n')
queryTokens = queryTokenize(query)
queryVector = vectorizer.transform([' '.join(queryTokens)])
simScores = cosine_similarity(tfidfs, queryVector)

bestFirstList = []
for i in range(len(cleanWebData)):
    bestFirstList.append((simScores[i][0], i))

bestFirstList = sorted(bestFirstList, key= lambda x: x[0], reverse=True)

searcher = True
toFetch = 10

while searcher or morePages.lower() in {'y', 'yes'}:
    searcher = False
    correlateTitleURL(toFetch)
    morePages = str(input("\nFetch more pages?"))
    toFetch += 10

Enter a search query: computer science



1 -computing | UIC Today
https://today.uic.edu/tag/computing

2 -UIC aims to break through gender gap in computer science | UIC Today
https://today.uic.edu/uic-aims-to-break-through-gender-gap-in-computer-science

3 -computer science | UIC Today
https://today.uic.edu/tag/computer-science/page/2

4 -computer science | UIC Today
https://today.uic.edu/tag/computer-science

5 -computer science | UIC Today
https://today.uic.edu/tag/computer-science/page/3

6 -computer science | UIC Today
https://today.uic.edu/tag/computer-science/page/4

7 -UIC receives ongoing funding to boost diversity in computer science | UIC Today
https://today.uic.edu/uic-receives-ongoing-funding-to-boost-diversity-in-computer-science

8 - Computer Science | University of Illinois at Chicago 
https://cs.uic.edu

9 -NSF awards UIC $1.5M for new data science institute | UIC Today
https://today.uic.edu/nsf-awards-uic-1-5m-for-new-data-science-institute

10 -UIC offers new data sc

In [30]:
# pageTokens = {}
# invertedIndex = {}
# key = 1

# for item in cleanWebData:
#     temp = item.split()
#     pageTokens[key] = temp
    
#     for token in temp:
#         frequency = invertedIndex.setdefault(token, {}).get(key, 0)
#         invertedIndex.setdefault(token, {})[key] = frequency + 1
#     key += 1

# mostFreq = {}

# for page in pageTokens:
#     mostFreq[page] = Counter(pageTokens[page]).most_common(1)[0][1]

# idf = {}
# num = len(crawledList)

# for key in invertedIndex.keys():
#     df = len(invertedIndex[key].keys())
#     idf[key] = np.log2(num/df)
    
# tfIdf = copy.deepcopy(invertedIndex)

# for token in tfIdf:
#     for entry in tfIdf[token]:
#         tf = tfIdf[token][entry] / mostFreq[entry]
#         tfIdf[token][entry] = tf * idf[token]
        
# def documentLength(doc, tokens):
#     documentLen = 0
    
#     for token in set(tokens):
#         documentLen += tfIdf[token][doc] ** 2
    
#     documentLen = np.sqrt(documentLen)
#     return documentLen

# webpageLengths = {}

# for page in pageTokens:
#     webpageLengths[page] = documentLength(page, pageTokens[page])
    
# def queryTokenize(query):
#     temp = query.lower()
#     temp = temp.split()
#     temp = [ps.stem(word) for word in temp]
#     temp = [item for item in temp if item not in stop_list]
#     temp = [item for item in temp if len(item) > 2]
#     temp = (" ").join(temp)
#     for i in punctuation:
#         temp = temp.replace(i, "")
#     temp = re.sub("\d", "", temp)
#     return temp.split()

# def cosineSimilarity(documents, query):
#     similarityScores = {}
#     queryLen = 0
#     queryWeights = {}
    
#     queryDict = Counter(query)
    
#     for token in queryDict.keys():
#         tf = queryDict[token] / queryDict.most_common(1)[0][1]
#         queryWeights[token] = tf * idf.get(token, 0)
#         queryLen += queryWeights[token] ** 2
    
#     queryLen = np.sqrt(queryLen)
    
#     for word in query:
#         wordWeight = queryWeights.get(word)
        
#         if wordWeight:
#             for page in tfIdf[word].keys():
#                 similarityScores[page] = similarityScores.get(page, 0) + (tfIdf[word][page] * wordWeight)
    
#     for page in similarityScores:
#         similarityScores[page] = similarityScores[page] / (webpageLengths[page] * queryLen)
        
#     return sorted(similarityScores.items(), key= lambda x: x[1], reverse=True)